# Deep Learning model

In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [3]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth', 
                        'koi_prad', 'koi_teq', 'koi_insol', 'koi_model_snr', 'koi_tce_plnt_num', 
                        'koi_steff', 'koi_slogg', 'koi_srad', 'ra', 'dec', 'koi_kepmag']]
X = selected_features.values
print(X.shape)

(6991, 16)


# Create a Train Test Split

Use `koi_disposition` for the y values

In [4]:
# prepare y values
y = df['koi_disposition'].values

# Step 1: Label-encode data set
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

# Step 2: One-hot encoding
from keras.utils import to_categorical
one_hot_y = to_categorical(encoded_y)
one_hot_y


array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, one_hot_y, random_state=1)

In [6]:
X_train

array([[1.05484133e+01, 1.39064020e+02, 1.01700000e+00, ...,
        2.98095430e+02, 4.47370610e+01, 1.32040000e+01],
       [2.47543849e+01, 1.40207320e+02, 7.09000000e-01, ...,
        2.95735350e+02, 4.25762480e+01, 1.55140000e+01],
       [1.05733568e+00, 1.31792007e+02, 2.62000000e-01, ...,
        2.92184170e+02, 4.93100400e+01, 1.54140000e+01],
       ...,
       [1.07027283e+00, 1.31835891e+02, 1.26900000e+00, ...,
        2.99170620e+02, 4.08710590e+01, 1.59260000e+01],
       [1.87464901e+01, 1.47130740e+02, 5.08000000e-01, ...,
        2.88164700e+02, 4.58165090e+01, 1.58430000e+01],
       [2.10907909e+01, 1.33963160e+02, 8.32000000e-01, ...,
        2.85475040e+02, 4.80349310e+01, 1.42230000e+01]])

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [7]:
# Scale your data
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

In [8]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# no need to scale y since it is already categorical
y_train

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

# Train the Model



In [10]:
# define the model
from tensorflow.keras.models import Sequential
model = Sequential()

from tensorflow.keras.layers import Dense
number_inputs = 16
number_hidden_nodes = 6
model.add(Dense(units=number_hidden_nodes, activation='relu', input_dim=number_inputs))
number_classes = 3
model.add(Dense(units=number_classes, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 102       
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 21        
Total params: 123
Trainable params: 123
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Train the model
model.fit(X_train_scaled, y_train, epochs=1000, shuffle=True, verbose=2)

Epoch 1/1000
164/164 - 0s - loss: 1.0954 - accuracy: 0.4381
Epoch 2/1000
164/164 - 0s - loss: 0.9585 - accuracy: 0.5236
Epoch 3/1000
164/164 - 0s - loss: 0.9127 - accuracy: 0.5527
Epoch 4/1000
164/164 - 0s - loss: 0.8867 - accuracy: 0.5705
Epoch 5/1000
164/164 - 0s - loss: 0.8690 - accuracy: 0.5812
Epoch 6/1000
164/164 - 0s - loss: 0.8552 - accuracy: 0.5939
Epoch 7/1000
164/164 - 0s - loss: 0.8436 - accuracy: 0.6027
Epoch 8/1000
164/164 - 0s - loss: 0.8335 - accuracy: 0.6111
Epoch 9/1000
164/164 - 0s - loss: 0.8235 - accuracy: 0.6157
Epoch 10/1000
164/164 - 0s - loss: 0.8145 - accuracy: 0.6212
Epoch 11/1000
164/164 - 0s - loss: 0.8065 - accuracy: 0.6246
Epoch 12/1000
164/164 - 0s - loss: 0.7990 - accuracy: 0.6296
Epoch 13/1000
164/164 - 0s - loss: 0.7911 - accuracy: 0.6386
Epoch 14/1000
164/164 - 0s - loss: 0.7848 - accuracy: 0.6439
Epoch 15/1000
164/164 - 0s - loss: 0.7793 - accuracy: 0.6471
Epoch 16/1000
164/164 - 0s - loss: 0.7736 - accuracy: 0.6475
Epoch 17/1000
164/164 - 0s - loss

Epoch 135/1000
164/164 - 0s - loss: 0.6785 - accuracy: 0.7042
Epoch 136/1000
164/164 - 0s - loss: 0.6782 - accuracy: 0.7038
Epoch 137/1000
164/164 - 0s - loss: 0.6783 - accuracy: 0.7040
Epoch 138/1000
164/164 - 0s - loss: 0.6787 - accuracy: 0.7025
Epoch 139/1000
164/164 - 0s - loss: 0.6783 - accuracy: 0.6996
Epoch 140/1000
164/164 - 0s - loss: 0.6779 - accuracy: 0.7007
Epoch 141/1000
164/164 - 0s - loss: 0.6775 - accuracy: 0.7017
Epoch 142/1000
164/164 - 0s - loss: 0.6775 - accuracy: 0.7021
Epoch 143/1000
164/164 - 0s - loss: 0.6766 - accuracy: 0.7040
Epoch 144/1000
164/164 - 0s - loss: 0.6769 - accuracy: 0.7044
Epoch 145/1000
164/164 - 0s - loss: 0.6767 - accuracy: 0.7028
Epoch 146/1000
164/164 - 0s - loss: 0.6768 - accuracy: 0.7013
Epoch 147/1000
164/164 - 0s - loss: 0.6765 - accuracy: 0.7055
Epoch 148/1000
164/164 - 0s - loss: 0.6760 - accuracy: 0.7038
Epoch 149/1000
164/164 - 0s - loss: 0.6763 - accuracy: 0.7023
Epoch 150/1000
164/164 - 0s - loss: 0.6756 - accuracy: 0.7028
Epoch 15

164/164 - 0s - loss: 0.6523 - accuracy: 0.7179
Epoch 268/1000
164/164 - 0s - loss: 0.6527 - accuracy: 0.7137
Epoch 269/1000
164/164 - 0s - loss: 0.6521 - accuracy: 0.7204
Epoch 270/1000
164/164 - 0s - loss: 0.6521 - accuracy: 0.7145
Epoch 271/1000
164/164 - 0s - loss: 0.6515 - accuracy: 0.7191
Epoch 272/1000
164/164 - 0s - loss: 0.6519 - accuracy: 0.7158
Epoch 273/1000
164/164 - 0s - loss: 0.6511 - accuracy: 0.7149
Epoch 274/1000
164/164 - 0s - loss: 0.6511 - accuracy: 0.7185
Epoch 275/1000
164/164 - 0s - loss: 0.6506 - accuracy: 0.7168
Epoch 276/1000
164/164 - 0s - loss: 0.6507 - accuracy: 0.7171
Epoch 277/1000
164/164 - 0s - loss: 0.6502 - accuracy: 0.7168
Epoch 278/1000
164/164 - 0s - loss: 0.6505 - accuracy: 0.7166
Epoch 279/1000
164/164 - 0s - loss: 0.6512 - accuracy: 0.7173
Epoch 280/1000
164/164 - 0s - loss: 0.6493 - accuracy: 0.7208
Epoch 281/1000
164/164 - 0s - loss: 0.6500 - accuracy: 0.7175
Epoch 282/1000
164/164 - 0s - loss: 0.6492 - accuracy: 0.7196
Epoch 283/1000
164/164 

Epoch 400/1000
164/164 - 0s - loss: 0.6291 - accuracy: 0.7326
Epoch 401/1000
164/164 - 0s - loss: 0.6288 - accuracy: 0.7353
Epoch 402/1000
164/164 - 0s - loss: 0.6282 - accuracy: 0.7376
Epoch 403/1000
164/164 - 0s - loss: 0.6287 - accuracy: 0.7328
Epoch 404/1000
164/164 - 0s - loss: 0.6288 - accuracy: 0.7320
Epoch 405/1000
164/164 - 0s - loss: 0.6284 - accuracy: 0.7341
Epoch 406/1000
164/164 - 0s - loss: 0.6286 - accuracy: 0.7347
Epoch 407/1000
164/164 - 0s - loss: 0.6281 - accuracy: 0.7326
Epoch 408/1000
164/164 - 0s - loss: 0.6286 - accuracy: 0.7381
Epoch 409/1000
164/164 - 0s - loss: 0.6281 - accuracy: 0.7364
Epoch 410/1000
164/164 - 0s - loss: 0.6274 - accuracy: 0.7366
Epoch 411/1000
164/164 - 0s - loss: 0.6274 - accuracy: 0.7343
Epoch 412/1000
164/164 - 0s - loss: 0.6276 - accuracy: 0.7366
Epoch 413/1000
164/164 - 0s - loss: 0.6270 - accuracy: 0.7360
Epoch 414/1000
164/164 - 0s - loss: 0.6277 - accuracy: 0.7339
Epoch 415/1000
164/164 - 0s - loss: 0.6282 - accuracy: 0.7355
Epoch 41

164/164 - 0s - loss: 0.6194 - accuracy: 0.7458
Epoch 533/1000
164/164 - 0s - loss: 0.6178 - accuracy: 0.7448
Epoch 534/1000
164/164 - 0s - loss: 0.6189 - accuracy: 0.7435
Epoch 535/1000
164/164 - 0s - loss: 0.6186 - accuracy: 0.7482
Epoch 536/1000
164/164 - 0s - loss: 0.6203 - accuracy: 0.7444
Epoch 537/1000
164/164 - 0s - loss: 0.6193 - accuracy: 0.7442
Epoch 538/1000
164/164 - 0s - loss: 0.6182 - accuracy: 0.7452
Epoch 539/1000
164/164 - 0s - loss: 0.6187 - accuracy: 0.7459
Epoch 540/1000
164/164 - 0s - loss: 0.6184 - accuracy: 0.7465
Epoch 541/1000
164/164 - 0s - loss: 0.6181 - accuracy: 0.7437
Epoch 542/1000
164/164 - 0s - loss: 0.6182 - accuracy: 0.7459
Epoch 543/1000
164/164 - 0s - loss: 0.6188 - accuracy: 0.7425
Epoch 544/1000
164/164 - 0s - loss: 0.6190 - accuracy: 0.7435
Epoch 545/1000
164/164 - 0s - loss: 0.6182 - accuracy: 0.7419
Epoch 546/1000
164/164 - 0s - loss: 0.6187 - accuracy: 0.7477
Epoch 547/1000
164/164 - 0s - loss: 0.6174 - accuracy: 0.7404
Epoch 548/1000
164/164 

Epoch 665/1000
164/164 - 0s - loss: 0.6115 - accuracy: 0.7519
Epoch 666/1000
164/164 - 0s - loss: 0.6116 - accuracy: 0.7482
Epoch 667/1000
164/164 - 0s - loss: 0.6115 - accuracy: 0.7522
Epoch 668/1000
164/164 - 0s - loss: 0.6116 - accuracy: 0.7496
Epoch 669/1000
164/164 - 0s - loss: 0.6114 - accuracy: 0.7467
Epoch 670/1000
164/164 - 0s - loss: 0.6120 - accuracy: 0.7454
Epoch 671/1000
164/164 - 0s - loss: 0.6123 - accuracy: 0.7477
Epoch 672/1000
164/164 - 0s - loss: 0.6106 - accuracy: 0.7484
Epoch 673/1000
164/164 - 0s - loss: 0.6125 - accuracy: 0.7484
Epoch 674/1000
164/164 - 0s - loss: 0.6107 - accuracy: 0.7461
Epoch 675/1000
164/164 - 0s - loss: 0.6119 - accuracy: 0.7492
Epoch 676/1000
164/164 - 0s - loss: 0.6105 - accuracy: 0.7538
Epoch 677/1000
164/164 - 0s - loss: 0.6106 - accuracy: 0.7513
Epoch 678/1000
164/164 - 0s - loss: 0.6111 - accuracy: 0.7505
Epoch 679/1000
164/164 - 0s - loss: 0.6103 - accuracy: 0.7480
Epoch 680/1000
164/164 - 0s - loss: 0.6114 - accuracy: 0.7503
Epoch 68

164/164 - 0s - loss: 0.6058 - accuracy: 0.7507
Epoch 798/1000
164/164 - 0s - loss: 0.6067 - accuracy: 0.7522
Epoch 799/1000
164/164 - 0s - loss: 0.6057 - accuracy: 0.7519
Epoch 800/1000
164/164 - 0s - loss: 0.6054 - accuracy: 0.7513
Epoch 801/1000
164/164 - 0s - loss: 0.6066 - accuracy: 0.7532
Epoch 802/1000
164/164 - 0s - loss: 0.6064 - accuracy: 0.7488
Epoch 803/1000
164/164 - 0s - loss: 0.6063 - accuracy: 0.7505
Epoch 804/1000
164/164 - 0s - loss: 0.6062 - accuracy: 0.7524
Epoch 805/1000
164/164 - 0s - loss: 0.6056 - accuracy: 0.7505
Epoch 806/1000
164/164 - 0s - loss: 0.6062 - accuracy: 0.7507
Epoch 807/1000
164/164 - 0s - loss: 0.6047 - accuracy: 0.7532
Epoch 808/1000
164/164 - 0s - loss: 0.6076 - accuracy: 0.7494
Epoch 809/1000
164/164 - 0s - loss: 0.6060 - accuracy: 0.7513
Epoch 810/1000
164/164 - 0s - loss: 0.6064 - accuracy: 0.7515
Epoch 811/1000
164/164 - 0s - loss: 0.6055 - accuracy: 0.7536
Epoch 812/1000
164/164 - 0s - loss: 0.6058 - accuracy: 0.7534
Epoch 813/1000
164/164 

Epoch 930/1000
164/164 - 0s - loss: 0.6009 - accuracy: 0.7543
Epoch 931/1000
164/164 - 0s - loss: 0.6038 - accuracy: 0.7500
Epoch 932/1000
164/164 - 0s - loss: 0.6011 - accuracy: 0.7526
Epoch 933/1000
164/164 - 0s - loss: 0.6017 - accuracy: 0.7561
Epoch 934/1000
164/164 - 0s - loss: 0.6026 - accuracy: 0.7538
Epoch 935/1000
164/164 - 0s - loss: 0.6011 - accuracy: 0.7538
Epoch 936/1000
164/164 - 0s - loss: 0.6024 - accuracy: 0.7526
Epoch 937/1000
164/164 - 0s - loss: 0.6015 - accuracy: 0.7530
Epoch 938/1000
164/164 - 0s - loss: 0.6015 - accuracy: 0.7528
Epoch 939/1000
164/164 - 0s - loss: 0.6011 - accuracy: 0.7515
Epoch 940/1000
164/164 - 0s - loss: 0.6022 - accuracy: 0.7549
Epoch 941/1000
164/164 - 0s - loss: 0.6010 - accuracy: 0.7557
Epoch 942/1000
164/164 - 0s - loss: 0.6019 - accuracy: 0.7541
Epoch 943/1000
164/164 - 0s - loss: 0.6024 - accuracy: 0.7521
Epoch 944/1000
164/164 - 0s - loss: 0.6019 - accuracy: 0.7561
Epoch 945/1000
164/164 - 0s - loss: 0.6038 - accuracy: 0.7561
Epoch 94

In [13]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
# print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
# print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

55/55 - 0s - loss: 0.6042 - accuracy: 0.7529
Loss: 0.604225754737854, Accuracy: 0.752860426902771


## Deep Learning

In [14]:
# Deep learning
deep_model = Sequential()
deep_model.add(Dense(units=6, activation='relu', input_dim=number_inputs))
deep_model.add(Dense(units=6, activation='relu'))
deep_model.add(Dense(units=number_classes, activation='softmax'))
deep_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 21        
Total params: 165
Trainable params: 165
Non-trainable params: 0
_________________________________________________________________


In [15]:
deep_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
deep_model.fit(X_train_scaled, y_train, epochs=1000, shuffle=True, verbose=2)

Epoch 1/1000
164/164 - 0s - loss: 1.0670 - accuracy: 0.4391
Epoch 2/1000
164/164 - 0s - loss: 0.9674 - accuracy: 0.5186
Epoch 3/1000
164/164 - 0s - loss: 0.9218 - accuracy: 0.5428
Epoch 4/1000
164/164 - 0s - loss: 0.8881 - accuracy: 0.5720
Epoch 5/1000
164/164 - 0s - loss: 0.8492 - accuracy: 0.6021
Epoch 6/1000
164/164 - 0s - loss: 0.8087 - accuracy: 0.6258
Epoch 7/1000
164/164 - 0s - loss: 0.7870 - accuracy: 0.6435
Epoch 8/1000
164/164 - 0s - loss: 0.7754 - accuracy: 0.6481
Epoch 9/1000
164/164 - 0s - loss: 0.7677 - accuracy: 0.6496
Epoch 10/1000
164/164 - 0s - loss: 0.7606 - accuracy: 0.6546
Epoch 11/1000
164/164 - 0s - loss: 0.7538 - accuracy: 0.6584
Epoch 12/1000
164/164 - 0s - loss: 0.7480 - accuracy: 0.6592
Epoch 13/1000
164/164 - 0s - loss: 0.7426 - accuracy: 0.6664
Epoch 14/1000
164/164 - 0s - loss: 0.7382 - accuracy: 0.6683
Epoch 15/1000
164/164 - 0s - loss: 0.7335 - accuracy: 0.6708
Epoch 16/1000
164/164 - 0s - loss: 0.7287 - accuracy: 0.6676
Epoch 17/1000
164/164 - 0s - loss

Epoch 135/1000
164/164 - 0s - loss: 0.6514 - accuracy: 0.7078
Epoch 136/1000
164/164 - 0s - loss: 0.6518 - accuracy: 0.7072
Epoch 137/1000
164/164 - 0s - loss: 0.6505 - accuracy: 0.7080
Epoch 138/1000
164/164 - 0s - loss: 0.6497 - accuracy: 0.7082
Epoch 139/1000
164/164 - 0s - loss: 0.6499 - accuracy: 0.7095
Epoch 140/1000
164/164 - 0s - loss: 0.6505 - accuracy: 0.7099
Epoch 141/1000
164/164 - 0s - loss: 0.6499 - accuracy: 0.7093
Epoch 142/1000
164/164 - 0s - loss: 0.6497 - accuracy: 0.7078
Epoch 143/1000
164/164 - 0s - loss: 0.6491 - accuracy: 0.7076
Epoch 144/1000
164/164 - 0s - loss: 0.6501 - accuracy: 0.7074
Epoch 145/1000
164/164 - 0s - loss: 0.6497 - accuracy: 0.7103
Epoch 146/1000
164/164 - 0s - loss: 0.6491 - accuracy: 0.7088
Epoch 147/1000
164/164 - 0s - loss: 0.6489 - accuracy: 0.7076
Epoch 148/1000
164/164 - 0s - loss: 0.6482 - accuracy: 0.7088
Epoch 149/1000
164/164 - 0s - loss: 0.6494 - accuracy: 0.7093
Epoch 150/1000
164/164 - 0s - loss: 0.6494 - accuracy: 0.7068
Epoch 15

164/164 - 0s - loss: 0.6378 - accuracy: 0.7177
Epoch 268/1000
164/164 - 0s - loss: 0.6358 - accuracy: 0.7183
Epoch 269/1000
164/164 - 0s - loss: 0.6361 - accuracy: 0.7196
Epoch 270/1000
164/164 - 0s - loss: 0.6365 - accuracy: 0.7166
Epoch 271/1000
164/164 - 0s - loss: 0.6359 - accuracy: 0.7181
Epoch 272/1000
164/164 - 0s - loss: 0.6356 - accuracy: 0.7204
Epoch 273/1000
164/164 - 0s - loss: 0.6354 - accuracy: 0.7185
Epoch 274/1000
164/164 - 0s - loss: 0.6362 - accuracy: 0.7204
Epoch 275/1000
164/164 - 0s - loss: 0.6352 - accuracy: 0.7210
Epoch 276/1000
164/164 - 0s - loss: 0.6344 - accuracy: 0.7198
Epoch 277/1000
164/164 - 0s - loss: 0.6335 - accuracy: 0.7242
Epoch 278/1000
164/164 - 0s - loss: 0.6340 - accuracy: 0.7276
Epoch 279/1000
164/164 - 0s - loss: 0.6341 - accuracy: 0.7198
Epoch 280/1000
164/164 - 0s - loss: 0.6348 - accuracy: 0.7173
Epoch 281/1000
164/164 - 0s - loss: 0.6333 - accuracy: 0.7242
Epoch 282/1000
164/164 - 0s - loss: 0.6339 - accuracy: 0.7215
Epoch 283/1000
164/164 

Epoch 400/1000
164/164 - 0s - loss: 0.5992 - accuracy: 0.7488
Epoch 401/1000
164/164 - 0s - loss: 0.5985 - accuracy: 0.7435
Epoch 402/1000
164/164 - 0s - loss: 0.5968 - accuracy: 0.7469
Epoch 403/1000
164/164 - 0s - loss: 0.5983 - accuracy: 0.7448
Epoch 404/1000
164/164 - 0s - loss: 0.5980 - accuracy: 0.7467
Epoch 405/1000
164/164 - 0s - loss: 0.5962 - accuracy: 0.7503
Epoch 406/1000
164/164 - 0s - loss: 0.5970 - accuracy: 0.7503
Epoch 407/1000
164/164 - 0s - loss: 0.5975 - accuracy: 0.7496
Epoch 408/1000
164/164 - 0s - loss: 0.5958 - accuracy: 0.7488
Epoch 409/1000
164/164 - 0s - loss: 0.5961 - accuracy: 0.7494
Epoch 410/1000
164/164 - 0s - loss: 0.5977 - accuracy: 0.7515
Epoch 411/1000
164/164 - 0s - loss: 0.5966 - accuracy: 0.7475
Epoch 412/1000
164/164 - 0s - loss: 0.5958 - accuracy: 0.7528
Epoch 413/1000
164/164 - 0s - loss: 0.5952 - accuracy: 0.7484
Epoch 414/1000
164/164 - 0s - loss: 0.5946 - accuracy: 0.7498
Epoch 415/1000
164/164 - 0s - loss: 0.5953 - accuracy: 0.7452
Epoch 41

164/164 - 0s - loss: 0.5843 - accuracy: 0.7568
Epoch 533/1000
164/164 - 0s - loss: 0.5855 - accuracy: 0.7547
Epoch 534/1000
164/164 - 0s - loss: 0.5838 - accuracy: 0.7593
Epoch 535/1000
164/164 - 0s - loss: 0.5845 - accuracy: 0.7551
Epoch 536/1000
164/164 - 0s - loss: 0.5848 - accuracy: 0.7557
Epoch 537/1000
164/164 - 0s - loss: 0.5845 - accuracy: 0.7555
Epoch 538/1000
164/164 - 0s - loss: 0.5846 - accuracy: 0.7583
Epoch 539/1000
164/164 - 0s - loss: 0.5859 - accuracy: 0.7568
Epoch 540/1000
164/164 - 0s - loss: 0.5839 - accuracy: 0.7576
Epoch 541/1000
164/164 - 0s - loss: 0.5843 - accuracy: 0.7585
Epoch 542/1000
164/164 - 0s - loss: 0.5844 - accuracy: 0.7574
Epoch 543/1000
164/164 - 0s - loss: 0.5836 - accuracy: 0.7578
Epoch 544/1000
164/164 - 0s - loss: 0.5825 - accuracy: 0.7589
Epoch 545/1000
164/164 - 0s - loss: 0.5850 - accuracy: 0.7601
Epoch 546/1000
164/164 - 0s - loss: 0.5837 - accuracy: 0.7595
Epoch 547/1000
164/164 - 0s - loss: 0.5839 - accuracy: 0.7595
Epoch 548/1000
164/164 

Epoch 665/1000
164/164 - 0s - loss: 0.5836 - accuracy: 0.7559
Epoch 666/1000
164/164 - 0s - loss: 0.5805 - accuracy: 0.7633
Epoch 667/1000
164/164 - 0s - loss: 0.5808 - accuracy: 0.7608
Epoch 668/1000
164/164 - 0s - loss: 0.5844 - accuracy: 0.7582
Epoch 669/1000
164/164 - 0s - loss: 0.5826 - accuracy: 0.7587
Epoch 670/1000
164/164 - 0s - loss: 0.5821 - accuracy: 0.7570
Epoch 671/1000
164/164 - 0s - loss: 0.5817 - accuracy: 0.7582
Epoch 672/1000
164/164 - 0s - loss: 0.5802 - accuracy: 0.7566
Epoch 673/1000
164/164 - 0s - loss: 0.5804 - accuracy: 0.7589
Epoch 674/1000
164/164 - 0s - loss: 0.5801 - accuracy: 0.7593
Epoch 675/1000
164/164 - 0s - loss: 0.5800 - accuracy: 0.7587
Epoch 676/1000
164/164 - 0s - loss: 0.5813 - accuracy: 0.7574
Epoch 677/1000
164/164 - 0s - loss: 0.5813 - accuracy: 0.7614
Epoch 678/1000
164/164 - 0s - loss: 0.5808 - accuracy: 0.7604
Epoch 679/1000
164/164 - 0s - loss: 0.5823 - accuracy: 0.7572
Epoch 680/1000
164/164 - 0s - loss: 0.5816 - accuracy: 0.7555
Epoch 68

164/164 - 0s - loss: 0.5794 - accuracy: 0.7578
Epoch 798/1000
164/164 - 0s - loss: 0.5785 - accuracy: 0.7583
Epoch 799/1000
164/164 - 0s - loss: 0.5779 - accuracy: 0.7570
Epoch 800/1000
164/164 - 0s - loss: 0.5790 - accuracy: 0.7599
Epoch 801/1000
164/164 - 0s - loss: 0.5798 - accuracy: 0.7582
Epoch 802/1000
164/164 - 0s - loss: 0.5805 - accuracy: 0.7570
Epoch 803/1000
164/164 - 0s - loss: 0.5785 - accuracy: 0.7583
Epoch 804/1000
164/164 - 0s - loss: 0.5777 - accuracy: 0.7595
Epoch 805/1000
164/164 - 0s - loss: 0.5782 - accuracy: 0.7574
Epoch 806/1000
164/164 - 0s - loss: 0.5786 - accuracy: 0.7570
Epoch 807/1000
164/164 - 0s - loss: 0.5798 - accuracy: 0.7574
Epoch 808/1000
164/164 - 0s - loss: 0.5790 - accuracy: 0.7566
Epoch 809/1000
164/164 - 0s - loss: 0.5793 - accuracy: 0.7570
Epoch 810/1000
164/164 - 0s - loss: 0.5797 - accuracy: 0.7559
Epoch 811/1000
164/164 - 0s - loss: 0.5787 - accuracy: 0.7557
Epoch 812/1000
164/164 - 0s - loss: 0.5788 - accuracy: 0.7589
Epoch 813/1000
164/164 

Epoch 930/1000
164/164 - 0s - loss: 0.5757 - accuracy: 0.7597
Epoch 931/1000
164/164 - 0s - loss: 0.5763 - accuracy: 0.7576
Epoch 932/1000
164/164 - 0s - loss: 0.5776 - accuracy: 0.7595
Epoch 933/1000
164/164 - 0s - loss: 0.5766 - accuracy: 0.7585
Epoch 934/1000
164/164 - 0s - loss: 0.5767 - accuracy: 0.7587
Epoch 935/1000
164/164 - 0s - loss: 0.5779 - accuracy: 0.7595
Epoch 936/1000
164/164 - 0s - loss: 0.5764 - accuracy: 0.7576
Epoch 937/1000
164/164 - 0s - loss: 0.5766 - accuracy: 0.7589
Epoch 938/1000
164/164 - 0s - loss: 0.5783 - accuracy: 0.7541
Epoch 939/1000
164/164 - 0s - loss: 0.5752 - accuracy: 0.7591
Epoch 940/1000
164/164 - 0s - loss: 0.5764 - accuracy: 0.7599
Epoch 941/1000
164/164 - 0s - loss: 0.5785 - accuracy: 0.7551
Epoch 942/1000
164/164 - 0s - loss: 0.5781 - accuracy: 0.7585
Epoch 943/1000
164/164 - 0s - loss: 0.5761 - accuracy: 0.7616
Epoch 944/1000
164/164 - 0s - loss: 0.5764 - accuracy: 0.7574
Epoch 945/1000
164/164 - 0s - loss: 0.5782 - accuracy: 0.7568
Epoch 94

In [17]:
deep_model_loss, deep_model_accuracy = deep_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {deep_model_loss}, Accuracy: {deep_model_accuracy}")

55/55 - 0s - loss: 0.5961 - accuracy: 0.7523
Loss: 0.5961129069328308, Accuracy: 0.7522883415222168


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [18]:
# Create the GridSearchCV model

In [19]:
# Train the model with GridSearch

In [20]:
# print(grid2.best_params_)
# print(grid2.best_score_)

# Save the Model

In [21]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
# import joblib
# filename = 'jamesye.sav'
# joblib.dump(deep_model, filename)
model.save("deep_model.h5")